In [ ]:
import importlib
import subprocess
import sys
import gc

def check_and_install_package(package_name):
    try:
        importlib.import_module(package_name)
        print(f"\n{package_name} is already installed.")
    except ImportError:
        print(f"\n{package_name} is NOT installed. Installing now...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package_name])
        print(f"{package_name} installation completed.")

# List of packages to check
packages = [
    "tqdm",
    "dask",
    "nltk",
    "scikit-learn",
    "numpy",
    "pyspark",
    "gdown"
]

# Checking and installing the packages
for package in packages:
    check_and_install_package(package)



tqdm is already installed.

dask is already installed.

nltk is already installed.

scikit-learn is NOT installed. Installing now...
scikit-learn installation completed.

numpy is already installed.

pyspark is already installed.

gdown is already installed.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("RandomForestModel") \
    .config("spark.driver.memory", "16g") \
    .config("spark.executor.memory", "16g") \
    .config("spark.driver.maxResultSize", "8g") \
    .config("spark.executor.memoryOverhead", "12g") \
    .config("spark.executor.cores", "5") \
    .config("spark.kryoserializer.buffer.max", "2047m") \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .config("spark.dynamicAllocation.enabled", "true") \
    .config("spark.hadoop.fs.file.impl", "org.apache.hadoop.fs.LocalFileSystem") \
    .config("spark.executor.extraJavaOptions", "-XX:+UseG1GC -XX:InitiatingHeapOccupancyPercent=35 -XX:ConcGCThreads=4 -XX:ParallelGCThreads=4") \
    .getOrCreate()

# Verifying Spark session creation
print(f"Spark session started with version: {spark.version}")

Spark session started with version: 3.5.3


# **Predictions BEFORE Feature Engineering**

In [ ]:
# loading the df

!cp '/content/drive/MyDrive/Big Data Analytics - Project/Datasets/Processed_DF.parquet' /content/

output_path = '/content/Processed_DF.parquet'
df = spark.read.parquet(output_path)
print("The Processed DataFrame has been loaded successfully.")


The Processed DataFrame has been loaded successfully.


In [ ]:
df = df.repartition(100)  # Repartitioning into 100 partitions for parallelism

In [ ]:
# Printing the shape of the DataFrame
total_rows = df.count()
total_columns = len(df.columns)

print(f"The shape of the loaded DataFrame is: ({total_rows}, {total_columns})")

The shape of the loaded DataFrame is: (3000040, 42)


In [ ]:
from functools import reduce
from pyspark.sql.functions import col

# Checking for rows with at least one null value in any column
rows_with_null = df.filter(
    reduce(lambda x, y: x | y, (col(c).isNull() for c in df.columns))
).count()

print(f"Number of rows with at least one null value: {rows_with_null}")

Number of rows with at least one null value: 0


### **Handling Categorical Coloumns**

In [ ]:
df=df.drop('description','major_options','mileage')
# Keeping the columns ['exterior_color','dealer_zip','interior_color']

In [ ]:
# Counting unique values in 'exterior_color' and 'interior_color' columns
exterior_colors_count = df.select('exterior_color').distinct().count()
interior_colors_count = df.select('interior_color').distinct().count()

print(f"Unique exterior colors: {exterior_colors_count}")
print(f"Unique interior colors: {interior_colors_count}")

Unique exterior colors: 23036
Unique interior colors: 38528


In [ ]:
from pyspark.sql import functions as F
from pyspark.sql.types import ArrayType, StringType

# Listing top colors for both exterior and interior
colors = ['White', 'Black', 'Gray', 'Silver', 'Red', 'Blue', 'Brown', 'Green', 'Beige', 'Orange', 'Gold', 'Yellow', 'Purple']

# Creating a UDF to find colors in the color columns
@F.udf(returnType=ArrayType(StringType()))
def find_colors(color_string):
    if color_string is None or color_string.strip() == "":
        return ["Other"]  # Handle empty or null values
    found_colors = [c for c in colors if c.lower() in color_string.lower()]
    return found_colors if found_colors else ["Other"]  # Label non-matching colors as "Other"

# Applying the UDF to both the exterior and interior color columns
df = df.withColumn("exterior_color_array", find_colors("exterior_color"))
df = df.withColumn("interior_color_array", find_colors("interior_color"))

# Creating a column with the count of colors found for both exterior and interior
df = df.withColumn("exterior_color_count", F.size("exterior_color_array"))
df = df.withColumn("interior_color_count", F.size("interior_color_array"))

# Joining the color arrays into string columns
df = df.withColumn("exterior_color", F.array_join("exterior_color_array", ", "))
df = df.withColumn("interior_color", F.array_join("interior_color_array", ", "))

# Labeling mixed colors for both exterior and interior colors
df = df.withColumn(
    "exterior_color",
    F.when(F.col("exterior_color_count") > 1, "Mixed Colors")
     .otherwise(F.col("exterior_color")))

df = df.withColumn(
    "interior_color",
    F.when(F.col("interior_color_count") > 1, "Mixed Colors")
     .otherwise(F.col("interior_color")))

# Dropping temporary columns
df = df.drop("exterior_color_array", "exterior_color_count", "interior_color_array", "interior_color_count")


In [ ]:
# Counting the occurrences of each exterior and interior color and calculating percentages
exterior_color_counts = df.groupBy("exterior_color").count().withColumn(
    "percentage", F.round((F.col("count") / df.count()) * 100, 2))

interior_color_counts = df.groupBy("interior_color").count().withColumn(
    "percentage", F.round((F.col("count") / df.count()) * 100, 2))

# Showing the results
print("Exterior Color Distribution:")
exterior_color_counts.orderBy(F.desc("count")).show(truncate=False)

Exterior Color Distribution:
+--------------+------+----------+
|exterior_color|count |percentage|
+--------------+------+----------+
|White         |675979|22.53     |
|Black         |580148|19.34     |
|Other         |543638|18.12     |
|Silver        |384540|12.82     |
|Blue          |253263|8.44      |
|Red           |242331|8.08      |
|Gray          |231172|7.71      |
|Green         |23026 |0.77      |
|Mixed Colors  |19728 |0.66      |
|Brown         |12905 |0.43      |
|Orange        |11638 |0.39      |
|Gold          |10544 |0.35      |
|Beige         |5065  |0.17      |
|Yellow        |4855  |0.16      |
|Purple        |1208  |0.04      |
+--------------+------+----------+



In [ ]:
print("Interior Color Distribution:")
interior_color_counts.orderBy(F.desc("count")).show(truncate=False)

Interior Color Distribution:
+--------------+-------+----------+
|interior_color|count  |percentage|
+--------------+-------+----------+
|Black         |1624033|54.13     |
|Other         |577578 |19.25     |
|Gray          |383966 |12.8      |
|Mixed Colors  |171212 |5.71      |
|White         |91545  |3.05      |
|Brown         |65943  |2.2       |
|Red           |34117  |1.14      |
|Silver        |24124  |0.8       |
|Blue          |22828  |0.76      |
|Green         |2048   |0.07      |
|Gold          |1193   |0.04      |
|Orange        |1133   |0.04      |
|Yellow        |134    |0.0       |
|Purple        |121    |0.0       |
|Beige         |65     |0.0       |
+--------------+-------+----------+



In [ ]:
print(f"Final processed DataFrame used for the model has {df.count()} rows and {len(df.columns)} columns.")

Final processed DataFrame used for the model has 3000040 rows and 39 columns.


In [ ]:
# Calculating the average price
avg_price = df.agg({"price": "avg"}).collect()[0][0]
print(f"Average price of a car: {round(avg_price)}")

Average price of a car: 29933


In [ ]:
import pandas as pd
from IPython.display import display

# Converting the Spark DataFrame to a Pandas DataFrame and displaying the first 5 rows with all columns
pd.set_option('display.max_columns', None)
display(df.limit(5).toPandas())


,fuel_type,body_type,city,city_fuel_economy,daysonmarket,dealer_zip,engine_displacement,engine_type,exterior_color,franchise_dealer,fuel_tank_volume,height,highway_fuel_economy,horsepower,interior_color,is_new,latitude,length,listed_date,listing_color,longitude,make_name,maximum_seating,model_name,price,savings_amount,seller_rating,sp_name,torque,transmission,transmission_display,wheel_system_display,wheelbase,width,year,combined_fuel_economy,legroom,log_mileage,major_options_count
0,Gasoline,SUV / Crossover,Ontario,20.000000,23,91761,3500.0,V6,Blue,True,19.500000,70.6,27.000000,284.0,Black,True,34.048500,198.5,2020-08-20,BLUE,-117.545998,Nissan,7.0,Pathfinder,34090.0,0,4.200000,Empire Nissan,259.00,CVT,Continuously Variable Transmission,Front-Wheel Drive,114.2,77.3,2020,23.50,83.9,1.61,5
1,Gasoline,Sedan,Elizabeth,22.000000,22,07202,2000.0,I4,Black,False,19.799999,57.8,31.000000,252.0,Black,False,40.652599,194.2,2020-08-18,BLACK,-74.215897,Audi,5.0,A6,25905.0,849,3.485714,Where Luxury Meets,265.22,A,8-Speed Automatic,All-Wheel Drive,114.6,82.1,2017,26.50,78.7,10.30,8
2,Biodiesel,Pickup Truck,Omaha,22.690001,93,68134,6700.0,V8,White,True,34.000000,81.5,29.469999,475.0,Black,True,41.291599,250.0,2020-06-09,WHITE,-96.023804,Ford,6.0,F-250 Super Duty,72955.0,0,4.333333,Woodhouse Ford of Omaha,265.22,A,Automatic,Four-Wheel Drive,159.8,105.9,2020,26.08,87.5,1.39,5
3,Gasoline,SUV / Crossover,Clearwater,22.690001,163,33763,1500.0,I4,Black,True,14.900000,65.4,29.469999,170.0,Gray,True,28.003901,183.1,2020-03-31,BLACK,-82.729301,Chevrolet,5.0,Equinox,19285.0,0,4.265823,Dimmitt Chevrolet,203.00,A,6-Speed Automatic Overdrive,Front-Wheel Drive,107.3,72.6,2020,26.08,80.8,0.00,5
4,Gasoline,SUV / Crossover,Chillicothe,18.000000,25,64601,3000.0,V6,Blue,True,21.799999,70.2,24.000000,400.0,Black,False,39.784000,198.8,2020-08-16,BLUE,-93.552902,Ford,7.0,Explorer,53623.0,1540,3.849558,Woody's Automotive Group - Chillicothe,265.22,A,Automatic,All-Wheel Drive,119.1,89.3,2020,21.00,82.0,9.03,15


## **Random Forest Regressor**

In [ ]:
import warnings
from tqdm import tqdm
from pyspark.ml.feature import StringIndexer, VectorAssembler, StandardScaler, OneHotEncoder, Imputer
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.regression import RandomForestRegressor
from pyspark.sql.functions import mean as sql_mean, log
import pyspark.sql.functions as F
import time

# Ignore warnings
warnings.filterwarnings('ignore')

# Starting to track overall runtime
start_time = time.time()

with tqdm(total=7, desc="Processing and Training") as pbar:

    df_sample = df.sample(fraction=0.2, seed=42)  # Randomly sampling 20% of the data
    pbar.update(1)

    # Removing rows where 'price' is <= 0 (to avoid issues with log transformation)
    df_sample = df_sample.filter(F.col("price") > 0)

    # Log transforming the target variable
    df_sample = df_sample.withColumn("log_price", log("price"))
    pbar.update(1)

    # Handling categorical columns
    cat_columns = [field for (field, dtype) in df_sample.dtypes if dtype == "string"]
    stages = []
    for col_name in cat_columns:
        indexer = StringIndexer(inputCol=col_name, outputCol=f"{col_name}_indexed", handleInvalid="keep")
        encoder = OneHotEncoder(inputCol=f"{col_name}_indexed", outputCol=f"{col_name}_encoded")
        stages += [indexer, encoder]
    pbar.update(1)

    # Converting 'franchise_dealer' to numeric
    df_sample = df_sample.withColumn("franchise_dealer", F.col("franchise_dealer").cast("int"))

    # Assembling features (to ensure all columns used in 'VectorAssembler' are numeric)
    num_columns = [col for col in df_sample.columns if col not in ['price', 'log_price'] + cat_columns]
    encoded_columns = [f"{col}_encoded" for col in cat_columns]
    feature_columns = num_columns + encoded_columns
    assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
    stages += [assembler]
    pbar.update(1)

    # Adding scaling to the pipeline (to scale the assembled feature vectors)
    scaler = StandardScaler(inputCol="features", outputCol="scaled_features", withMean=True, withStd=True)
    stages += [scaler]

    # Creating and applying the pipeline
    pipeline = Pipeline(stages=stages)
    try:
        pipeline_model = pipeline.fit(df_sample)
        df_sample = pipeline_model.transform(df_sample)
        pbar.update(1)
    except Exception as e:
        print(f"Error during pipeline fit: {e}")
        pbar.update(1)

    # Splitting the data into training and test sets
    train_df, test_df = df_sample.randomSplit([0.8, 0.2], seed=42)
    pbar.update(1)

    # Defining the RandomForestRegressor model
    rf = RandomForestRegressor(
      featuresCol="scaled_features",
      labelCol="log_price",
      numTrees=50,
      maxDepth=10,
      minInstancesPerNode=10,
      seed=42
    )

    # Fitting the model to the training data
    try:
        model = rf.fit(train_df)
        pbar.update(1)
    except Exception as e:
        print(f"Error during model training: {e}")
        pbar.update(1)

# Making predictions
print("Making predictions...")
try:
    predictions = model.transform(test_df)
    predictions = predictions.withColumn("exp_prediction", F.exp("prediction"))
except Exception as e:
    print(f"Error during prediction: {e}")

# Evaluating the model
print("Evaluating the model...")
try:
    evaluator = RegressionEvaluator(labelCol="price", predictionCol="exp_prediction", metricName="r2")
    r2 = evaluator.evaluate(predictions)
    print(f"\n\nR-Squared Score (Accuracy): {round(r2 * 100)}%\n")
except Exception as e:
    print(f"Error during evaluation: {e}")

# Displaying results
print(f"\n\nTrain size: {train_df.count():,} samples")
print(f"Test size: {test_df.count():,} samples")

# Calculating total runtime
end_time = time.time()
total_runtime = (end_time - start_time) / 60
print(f"\nOverall runtime: {round(total_runtime)} minutes.")


Processing and Training: 100%|██████████| 7/7 [3:42:54<00:00, 1910.71s/it]
Making predictions...
Evaluating the model...


R-Squared Score (Accuracy): 78.10%


Train size: 480,538 samples
Test size: 120,299 samples

Overall runtime: 236 minutes.


In [ ]:
# Calculating additional metrics
mae_evaluator = RegressionEvaluator(labelCol="price", predictionCol="exp_prediction", metricName="mae")
mae = mae_evaluator.evaluate(predictions)

mse_evaluator = RegressionEvaluator(labelCol="price", predictionCol="exp_prediction", metricName="mse")
mse = mse_evaluator.evaluate(predictions)

rmse_evaluator = RegressionEvaluator(labelCol="price", predictionCol="exp_prediction", metricName="rmse")
rmse = rmse_evaluator.evaluate(predictions)

print("Additional Metrics:")
print(f"Mean Absolute Error: {round(mae)}")
print(f"Mean Squared Error: {round(mse)}")
print(f"Root Mean Squared Error: {round(rmse)}")

Additional Metrics:
Mean Absolute Error: 3791
Mean Squared Error: 83982448
Root Mean Squared Error: 9164


The Mean Absolute Error (`MAE`) of **\$3791**  suggests that, on average, the predicted car prices deviate from the actual prices by this amount. Given that the Mean car price is **\$28,334** , this error represents about `~12.6% of the mean price`, which suggests that the model performs very well.

In [ ]:
# Feature Importance (for Random Forest)
feature_importance = model.featureImportances
features_list = pipeline_model.stages[-2].getInputCols()  # Getting feature names from VectorAssembler

# Sorting features by their importance in descending order
sorted_features = [feature for feature, importance in sorted(zip(features_list, feature_importance), key=lambda x: x[1], reverse=True)]

# Printing ranked features from highest to lowest
print("Top 10 Features Ranked by Importance (Highest to Lowest):")
for rank, feature in enumerate(sorted_features[:10], 1):  # Limiting to top 10 features
    print(f"{rank}. {feature}")

Top 10 Features Ranked by Importance (Highest to Lowest):
1. log_mileage
2. year
3. horsepower
4. torque
5. is_new
6. width
7. savings_amount
8. fuel_tank_volume
9. franchise_dealer
10. engine_displacement


In [ ]:
import pyspark

# Getting the list of all variables in the current notebook environment
for var_name in dir():
    # Checking if the variable is a Spark DataFrame
    if isinstance(eval(var_name), pyspark.sql.dataframe.DataFrame):
        # Deleting the variables
        del globals()[var_name]

# Performing garbage collection
import gc
_ = gc.collect()
print("All DataFrames have been deleted.")


spark.catalog.clearCache()
spark.stop()
print("Spark Stopped !")

All DataFrames have been deleted.
Spark Stopped !




---



# **Predictions AFTER Feature Engineering**

In [ ]:
!cp '/content/drive/MyDrive/Big Data Analytics - Project/Datasets/Feature_Engineered_DF.parquet' /content/

output_path = '/content/Feature_Engineered_DF.parquet'
df = spark.read.parquet(output_path)
print("The Feature Engineered DataFrame has been loaded successfully.")


The Feature Engineered DataFrame has been loaded successfully.


In [ ]:
# Printing the shape of the DataFrame
total_rows = df.count()
total_columns = len(df.columns)

print(f"The shape of the loaded DataFrame is: ({total_rows}, {total_columns})")

The shape of the loaded DataFrame is: (3000040, 47)


In [ ]:
# Calculating the average price
avg_price = df.agg({"price": "avg"}).collect()[0][0]
print(f"Average price of a car: {round(avg_price)}")

Average price of a car: 29933


In [ ]:
import pandas as pd
from IPython.display import display

# Converting the Spark DataFrame to a Pandas DataFrame and displaying 5 random rows with all columns
pd.set_option('display.max_columns', None)
display(df.limit(5).toPandas())


,fuel_type,body_type,city,city_fuel_economy,days_in_market,dealer_zip,engine_displacement,engine_type,exterior_color,franchise_dealer,fuel_tank_volume,height,highway_fuel_economy,horsepower,interior_color,is_new,latitude,length,listing_color,longitude,make_name,maximum_seating,model_name,price,savings_amount,seller_rating,sp_name,torque,transmission,transmission_display,wheel_system_display,wheelbase,width,manufactured_year,combined_fuel_economy,legroom,log_mileage,major_options_count,hp_x_engine_disp,hp_x_torque,listed_day,listed_month,listed_year,age,resale_value_score,maintenance_cost,luxury_score
0,Gasoline,Sedan,San Antonio,30.000000,47,78233,1800.0,I4,Red,True,13.200000,56.5,38.000000,139.0,Other,True,29.549299,182.3,RED,-98.361801,Toyota,5.0,Corolla,22462.0,0,3.142857,Universal Toyota,265.22,CVT,Continuously Variable Transmission,Front-Wheel Drive,106.3,70.1,2020,34.00,76.8,8.91,7,1.07,-0.00001,26,7,2020,0,24,35,31
1,Gasoline,Sedan,Charlotte,17.000000,4,28273,3000.0,I6,White,True,15.800000,56.3,26.000000,425.0,Black,False,35.124699,183.9,WHITE,-80.878502,BMW,5.0,M3,51800.0,0,4.205128,Williams Buick GMC,406.00,M,6-Speed Manual,Rear-Wheel Drive,110.7,80.2,2017,21.50,77.1,10.49,7,0.09,2.61312,6,9,2020,3,29,43,32
2,Diesel,Pickup Truck,Waterbury,22.690001,8,06704,6400.0,V8,White,False,30.500000,80.6,29.469999,350.0,Gray,False,41.571999,231.8,WHITE,-73.026199,Ford,6.0,F-350 Super Duty,24995.0,4918,4.470588,Lakewood Auto,457.00,A,5-Speed Automatic,Four-Wheel Drive,141.8,99.6,2010,26.08,72.8,11.75,1,3.07,2.05484,1,9,2020,10,22,40,28
3,Gasoline,SUV / Crossover,Lewiston,20.000000,5,83501,2300.0,I4,Silver,True,19.200001,69.9,27.000000,300.0,Other,False,46.403500,198.8,SILVER,-117.001999,Ford,7.0,Explorer,41999.0,1359,4.000000,Joe Hall Ford Lincoln,265.22,A,Automatic,All-Wheel Drive,119.1,89.3,2020,23.50,82.0,8.06,5,-0.29,0.00001,6,9,2020,0,34,40,36
4,Gasoline,SUV / Crossover,Spring,21.000000,58,77388,2400.0,I4,Gray,True,18.799999,66.1,27.000000,185.0,Black,False,30.054899,187.8,GRAY,-95.477303,Hyundai,5.0,Santa Fe,20491.0,142,4.304348,AutoNation Chrysler Dodge Jeep Ram Spring,178.00,A,Automatic,All-Wheel Drive,108.9,74.4,2019,24.00,85.0,10.27,5,0.29,0.57120,15,7,2020,1,24,37,29


In [ ]:
import warnings
from tqdm import tqdm
from pyspark.ml.feature import StringIndexer, VectorAssembler, StandardScaler, OneHotEncoder, Imputer
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.regression import RandomForestRegressor
from pyspark.sql.functions import mean as sql_mean, log
import pyspark.sql.functions as F
import time

# Ignore warnings
warnings.filterwarnings('ignore')

# Start to track overall runtime
start_time = time.time()

with tqdm(total=7, desc="Processing and Training") as pbar:

    df_sample = df.sample(fraction=0.2, seed=42)  # Random sampling 20% of the data
    pbar.update(1)

    # Removing rows where 'price' is <= 0 (to avoid issues with log transformation)
    df_sample = df_sample.filter(F.col("price") > 0)

    # Log transforming the target variable
    df_sample = df_sample.withColumn("log_price", log("price"))
    pbar.update(1)

    # Handling categorical columns
    cat_columns = [field for (field, dtype) in df_sample.dtypes if dtype == "string"]
    stages = []
    for col_name in cat_columns:
        indexer = StringIndexer(inputCol=col_name, outputCol=f"{col_name}_indexed", handleInvalid="keep")
        encoder = OneHotEncoder(inputCol=f"{col_name}_indexed", outputCol=f"{col_name}_encoded")
        stages += [indexer, encoder]
    pbar.update(1)

    # Converting 'franchise_dealer' to numeric
    df_sample = df_sample.withColumn("franchise_dealer", F.col("franchise_dealer").cast("int"))

    # Assembling features (ensure all columns used in 'VectorAssembler' are numeric)
    num_columns = [col for col in df_sample.columns if col not in ['price', 'log_price'] + cat_columns]
    encoded_columns = [f"{col}_encoded" for col in cat_columns]
    feature_columns = num_columns + encoded_columns
    assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
    stages += [assembler]
    pbar.update(1)

    # Adding scaling to the pipeline (scaling the assembled feature vectors)
    scaler = StandardScaler(inputCol="features", outputCol="scaled_features", withMean=True, withStd=True)
    stages += [scaler]

    # Creating and appling the pipeline
    pipeline = Pipeline(stages=stages)
    try:
        pipeline_model = pipeline.fit(df_sample)
        df_sample = pipeline_model.transform(df_sample)
        pbar.update(1)
    except Exception as e:
        print(f"Error during pipeline fit: {e}")
        pbar.update(1)

    # Spliting the data into training and test sets
    train_df, test_df = df_sample.randomSplit([0.8, 0.2], seed=42)
    pbar.update(1)

    # Defining the RandomForestRegressor model
    rf = RandomForestRegressor(
      featuresCol="scaled_features",
      labelCol="log_price",
      numTrees=50,
      maxDepth=10,
      minInstancesPerNode=10,
      seed=42
    )

    # Fitting the model to the training data
    try:
        model = rf.fit(train_df)
        pbar.update(1)
    except Exception as e:
        print(f"Error during model training: {e}")
        pbar.update(1)

# Making predictions
print("Making predictions...")
try:
    predictions = model.transform(test_df)
    predictions = predictions.withColumn("exp_prediction", F.exp("prediction"))
except Exception as e:
    print(f"Error during prediction: {e}")

# Evaluating the model
print("Evaluating the model...")
try:
    evaluator = RegressionEvaluator(labelCol="price", predictionCol="exp_prediction", metricName="r2")
    r2 = evaluator.evaluate(predictions)
    print(f"\n\nR-Squared Score (Accuracy): {r2 * 100:.2f}%")
except Exception as e:
    print(f"Error during evaluation: {e}")

# Displaying results
print(f"\n\nTrain size: {train_df.count():,} samples")
print(f"Test size: {test_df.count():,} samples")

# Calculating total runtime
end_time = time.time()
total_runtime = (end_time - start_time) / 60
print(f"\nOverall runtime: {round(total_runtime)} minutes.")


Processing and Training: 100%|██████████| 7/7 [8:22:00<00:00, 4302.89s/it]


Making predictions...
Evaluating the model...


R-Squared Score (Accuracy): 86.35%


Train size: 480,411 samples
Test size: 120,366 samples

Overall runtime: 515 minutes.


In [ ]:
new_model_path = "/content/drive/MyDrive/Big Data Analytics - Project/models/Random_Forest_model_FE"

# Saving the trained model to the new path
model.save(new_model_path)

print(f"Model saved successfully at {new_model_path}")


Model saved successfully at /content/drive/MyDrive/Big Data Analytics - Project/models/Random_Forest_model_FE


In [ ]:
# Calculating additional metrics
mae_evaluator = RegressionEvaluator(labelCol="price", predictionCol="exp_prediction", metricName="mae")
mae = mae_evaluator.evaluate(predictions)

mse_evaluator = RegressionEvaluator(labelCol="price", predictionCol="exp_prediction", metricName="mse")
mse = mse_evaluator.evaluate(predictions)

rmse_evaluator = RegressionEvaluator(labelCol="price", predictionCol="exp_prediction", metricName="rmse")
rmse = rmse_evaluator.evaluate(predictions)

print("Additional Metrics:")
print(f"Mean Absolute Error: {round(mae)}")
print(f"Mean Squared Error: {round(mse)}")
print(f"Root Mean Squared Error: {round(rmse)}")

Additional Metrics:
Mean Absolute Error: 3488
Mean Squared Error: 44980420
Root Mean Squared Error: 6707


 In **Random Forest**, the model saw an **improvement in R² accuracy by 8.26%**, while the Mean Absolute Error **(MAE) was reduced by 1.01%**, and the Root Mean Square Error **(RMSE) was reduced by 8.21%**, demonstrating significantly enhanced predictive performance.

In [ ]:
# Feature Importance (for Random Forest)
feature_importance = model.featureImportances
features_list = pipeline_model.stages[-2].getInputCols()  # Getting feature names from VectorAssembler

# Sorting features by their importance in descending order
sorted_features = [feature for feature, importance in sorted(zip(features_list, feature_importance), key=lambda x: x[1], reverse=True)]

# Printing ranked features from highest to lowest
print("Top 10 Features Ranked by Importance (Highest to Lowest):")
for rank, feature in enumerate(sorted_features[:10], 1):  # Limiting to top 10 features
    print(f"{rank}. {feature}")

Top 10 Features Ranked by Importance (Highest to Lowest):
1. maintenance_cost
2. log_mileage
3. age
4. manufactured_year
5. horsepower
6. torque
7. luxury_score
8. resale_value_score
9. width
10. hp_x_torque
